# 1. Asset Allocation Recommendation Prototype

## Team members
### Lucas Sebastian A0112080B
### Sekson Ounsaengchan (Beer) A0227885M
### Zhao Mengyu (Jessica) A0227914B

## Objective

Create a prototype that receives ETF database (or any security), investment money as inputs and return asset allocation based on 4 investment themes:
1. Conservative portfolio (Fixed ratio, see detail below)
2. Aggressive portfolio (Fixed ratio, see detail below)
3. Minimum volatility portfolio
4. Maximum sharpe ratio portfolio


Program will calculate expected return, volatility, sharpe ratio, and stock quantities to purchase based on investment money. The front-end application will call this program and return asset allocation set that matches the investor's preference and risk profile.

<a id="section-two"></a>
# 2. SETUP

<a id="subsection-two-one"></a>
## 2.1 Install Yahoo Finance API

In [1]:
# Anaconda Prompt > "$ pip install yfinance --upgrade --no-cache-dir"
# https://pypi.org/project/fix-yahoo-finance/

<a id="subsection-two-two"></a>
## 2.2 Import/Wrangle Data

In [2]:
import pandas as pd  
import numpy as np
import datetime as dt 
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from dateutil.relativedelta import relativedelta

# visual styles
plt.style.use('fivethirtyeight')
np.random.seed(1)

See Other Visual Styles [HERE](https://matplotlib.org/3.2.1/gallery/style_sheets/style_sheets_reference.html)

In [3]:
ETF_DB = pd.read_csv(
    "C:/Users/sekso/Desktop/MBA/sem4/Fintech/Bootcamp Codes and Data/05 Monte Carlo Simulation/ETF_list.csv", na_values=["."]
)
# CSV file has 2 columns: Symbol and Asset type. Asset type is a security type that an ETF invests in. In this prototype, we have only Equity, Bonds, and Alternatives. See example below:

# Symbol,Asset type
# QAI,Alternatives
# MNA,Alternatives
# FTLS,Alternatives
# RLY,Alternatives
# WTMF,Alternatives
# AGG,Bond
# BND,Bond
# BNDX,Bond
# VCIT,Bond
# VCSH,Bond
# BSV,Bond
# LQD,Bond
# TIP,Bond
# SPY,Equity
# IVV,Equity
# VTI,Equity
# VOO,Equity
# QQQ,Equity
# VEA,Equity
# IEFA,Equity
# VTV,Equity
# VWO,Equity
# VUG,Equity

symbols_list = list(ETF_DB['Symbol'])
start = dt.datetime.now() - relativedelta(years=5) # 5 years ago
end = dt.datetime.now()
data = yf.download(symbols_list, start=start, end=end)

[*********************100%***********************]  23 of 23 completed


<a id="subsection-two-three"></a>
## 2.3 Inspect Data

In [4]:
data.head()

Adj Close                                                         \
                  AGG        BND       BNDX        BSV       FTLS       IEFA   
Date                                                                           
2017-03-06  95.824242  70.674477  47.299057  72.559219  34.198814  48.832268   
2017-03-07  95.699844  70.542984  47.281574  72.513565  34.062443  48.676613   
2017-03-08  95.468887  70.376457  47.150307  72.449646  34.003998  48.477726   
2017-03-09  95.175728  70.157379  47.054039  72.394913  33.828678  48.693905   
2017-03-10  95.317879  70.297585  46.957794  72.440521  33.867634  49.126282   

                                                          ...   Volume  \
                   IVV         LQD        MNA        QAI  ...      TIP   
Date                                                      ...            
2017-03-06  218.011169  100.878502  29.164543  27.315144  ...  1553600   
2017-03-07  217.373367  100.379692  29.184092  27.258648  ...  1545200   
2017-03-08  216.936005  100.070053  29.125448  27.239817  ...  1357200   
2017-03-09  217.145554   99.528191  29.144997  27.211571  ...  1098300   
2017-03-10  217.901825   99.777596  29.115677  27.286894  ...  2424500   

                                                                           \
               VCIT     VCSH       VEA      VOO      VTI      VTV     VUG   
Date                                                                        
2017-03-06   976300  1277100   6501700  1530600  1716200  1788300  877000   
2017-03-07  1042800  1439100   6354000  1743200  1612500  1388100  704400   
2017-03-08   758200  1102000   6088200  1784400  2374200  1489800  655100   
2017-03-09  1289300  1594400  14453500  1636200  3852800  1208400  755600   
2017-03-10   803700   944100  13367400  1819900  2987300  1360500  766000   

                             
                 VWO   WTMF  
Date                         
2017-03-06   8232800  20100  
2017-03-07  14789500  19800  
2017-03-08  15601800  20600  
2017-03-09  12131900  38800  
2017-03-10  14955600  75300  

[5 rows x 138 columns]

Let's focus on the variable adjusted close

In [5]:
maindata = data["Adj Close"]
maindata.head()

,AGG,BND,BNDX,BSV,FTLS,IEFA,IVV,LQD,MNA,QAI,...,TIP,VCIT,VCSH,VEA,VOO,VTI,VTV,VUG,VWO,WTMF
Date,,,,,,,,,,,,,,,,,,,,,
2017-03-06,95.824242,70.674477,47.299057,72.559219,34.198814,48.832268,218.011169,100.878502,29.164543,27.315144,...,100.320412,73.303200,70.650879,33.201546,199.385254,112.078484,85.347488,115.413528,34.156845,33.306290
2017-03-07,95.699844,70.542984,47.281574,72.513565,34.062443,48.676613,217.373367,100.379692,29.184092,27.258648,...,100.144035,73.089821,70.597549,33.115250,198.782425,111.720993,85.049011,115.108727,34.200634,33.256771
2017-03-08,95.468887,70.376457,47.150307,72.449646,34.003998,48.477726,216.936005,100.070053,29.125448,27.239817,...,99.711815,72.859329,70.526436,32.942631,198.362289,111.409332,84.838326,115.032516,33.859070,33.025707
2017-03-09,95.175728,70.157379,47.054039,72.394913,33.828678,48.693905,217.145554,99.528191,29.144997,27.211571,...,99.447220,72.534935,70.437546,33.080723,198.572388,111.464340,84.908539,115.089699,33.570053,32.901924
2017-03-10,95.317879,70.297585,46.957794,72.440521,33.867634,49.126282,217.901825,99.777596,29.115677,27.286894,...,99.632439,72.688576,70.473145,33.339642,199.266525,111.867676,85.180672,115.537369,33.850319,32.728626


In [6]:
maindata.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1259 entries, 2017-03-06 to 2022-03-03
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AGG     1259 non-null   float64
 1   BND     1259 non-null   float64
 2   BNDX    1259 non-null   float64
 3   BSV     1259 non-null   float64
 4   FTLS    1259 non-null   float64
 5   IEFA    1259 non-null   float64
 6   IVV     1259 non-null   float64
 7   LQD     1259 non-null   float64
 8   MNA     1259 non-null   float64
 9   QAI     1259 non-null   float64
 10  QQQ     1259 non-null   float64
 11  RLY     1259 non-null   float64
 12  SPY     1259 non-null   float64
 13  TIP     1259 non-null   float64
 14  VCIT    1259 non-null   float64
 15  VCSH    1259 non-null   float64
 16  VEA     1259 non-null   float64
 17  VOO     1259 non-null   float64
 18  VTI     1259 non-null   float64
 19  VTV     1259 non-null   float64
 20  VUG     1259 non-null   float64
 21  VWO     1259 non-nu

In [7]:
returns = maindata.pct_change()
returns

,AGG,BND,BNDX,BSV,FTLS,IEFA,IVV,LQD,MNA,QAI,...,TIP,VCIT,VCSH,VEA,VOO,VTI,VTV,VUG,VWO,WTMF
Date,,,,,,,,,,,,,,,,,,,,,
2017-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-07,-0.001298,-0.001861,-0.000370,-0.000629,-0.003988,-0.003188,-0.002926,-0.004945,0.000670,-0.002068,...,-0.001758,-0.002911,-0.000755,-0.002599,-0.003023,-0.003190,-0.003497,-0.002641,0.001282,-0.001487
2017-03-08,-0.002413,-0.002361,-0.002776,-0.000881,-0.001716,-0.004086,-0.002012,-0.003085,-0.002009,-0.000691,...,-0.004316,-0.003154,-0.001007,-0.005213,-0.002114,-0.002790,-0.002477,-0.000662,-0.009987,-0.006948
2017-03-09,-0.003071,-0.003113,-0.002042,-0.000755,-0.005156,0.004459,0.000966,-0.005415,0.000671,-0.001037,...,-0.002654,-0.004452,-0.001260,0.004192,0.001059,0.000494,0.000828,0.000497,-0.008536,-0.003748
2017-03-10,0.001494,0.001998,-0.002045,0.000630,0.001152,0.008879,0.003483,0.002506,-0.001006,0.002768,...,0.001862,0.002118,0.000505,0.007827,0.003496,0.003619,0.003205,0.003890,0.008349,-0.005267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-25,0.001005,0.000862,-0.001311,-0.000378,0.014282,0.025156,0.022016,0.004066,0.008192,0.010760,...,-0.000879,0.002044,-0.000126,0.025169,0.021840,0.022254,0.029830,0.015367,0.018530,-0.001428
2022-02-28,0.007210,0.007378,0.006753,0.003281,-0.007579,-0.017163,-0.002024,0.010690,-0.003438,-0.002581,...,0.017359,0.008044,0.003782,-0.014029,-0.002510,-0.001084,-0.005531,0.001842,-0.011371,0.000572
2022-03-01,0.005635,0.005245,0.011094,0.003475,0.000402,-0.021360,-0.015500,0.003420,0.000000,-0.003881,...,0.007160,0.004268,0.002037,-0.019669,-0.014999,-0.015599,-0.015016,-0.014818,-0.013593,0.016581


<a id="section-three"></a>
# 3. Construct portfolios based on 4 themes
1. Conservative portfolio
2. Aggressive portfolio
3. Minimum volatility portfolio
4. Maximum sharpe ratio portfolio

Prototype in this section could be used when individual investors want our system to suggest asset allocation when first registering with our platform. The application will call this section and return asset allocation set that matches the investor's preference and risk profile.

In [8]:
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    """
    Function returns the annualized performance given the weights of the portfolio. 
    Function assumes current means and cov are daily returns 
    
    Parameters:
        weights (numpy.array): array containing relative weights. 
        mean_returns (pandas.Series): mean return of each security. len(mean_returns) must match num_securities 
        cov_matrix (pandas.DataFrame): Covariance matrix of shape n * n, where n is num_securities 
    Returns:
        tuple: Tuple contains the standard deviation and returns of the portfolio, annualized. 
    """
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

Details on the calculation of portfolio standard deviation [HERE](https://stackoverflow.com/questions/59462628/is-there-a-way-to-vectorize-the-portfolio-standard-deviation-in-python-pandas)  
Details on dot product [HERE](https://towardsdatascience.com/linear-algebra-basics-dot-product-and-matrix-multiplication-2a7624942810).  


In [9]:
# Execute pip install PyPortfolioOpt in terminal first!
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

# Parameters setup
risk_free_rate = 0.02 # The 10-year treasury rate
Investment_money = 50000 # In US dollar


mean_returns = returns.mean()
sharpe_returns = (mean_returns*252 - risk_free_rate) / (returns.std()*np.sqrt(252))

# Set up dataframe containing asset allocation ratio between each type of assets. We used 2 themes as examples
#                Equity Bond Alternatives
# Conservative   0.2    0.2   0.3
# Aggressive     0.5    0.3   0.2
investment_theme = pd.DataFrame([[0.20,0.50,0.30],[0.50,0.30,0.20]],index=['conservative', 'aggressive'],columns=['Equity','Bond','Alternatives'])

# Select only highest sharpe ratio ETF for each asset type to be considered to construct the portfolio
sharpe_df = pd.DataFrame({'Symbol':sharpe_returns.index, 'Sharpe':sharpe_returns.values})
ETF_DB = pd.merge(sharpe_df, ETF_DB, on="Symbol")
ETF_DB = ETF_DB.sort_values(['Asset type','Sharpe'],ascending=False).groupby('Asset type').head(1)
symbols_list = list(ETF_DB['Symbol'])
ETF_DB.set_index('Symbol',inplace=True)
returns.drop([col for col in returns.columns if col not in symbols_list],axis=1,inplace=True)
maindata.drop([col for col in maindata.columns if col not in symbols_list],axis=1,inplace=True)
cov_matrix = returns.cov()
mean_returns = returns.mean()
num_securities = len(symbols_list)

# Assign weight to be used to calculate annualised return, standard deviation, and sharpe ratio
weight_conservative = np.array([0.0,0.0,0.0])
i = 0
for index, row in mean_returns.iteritems():
    weight_conservative[i] = investment_theme.loc['conservative'][ETF_DB.loc[index]['Asset type']]
    i += 1
    
weight_aggressive = np.array([0.0,0.0,0.0])
i = 0
for index, row in mean_returns.iteritems():
    weight_aggressive[i] = investment_theme.loc['aggressive'][ETF_DB.loc[index]['Asset type']]
    i += 1

# Calculation section
portfolio_con_std_dev, portfolio_con_return = portfolio_annualised_performance(weight_conservative, mean_returns, cov_matrix)
portfolio_con_sharpe = (portfolio_con_return - risk_free_rate) / portfolio_con_std_dev
con_allocation = pd.DataFrame(weight_conservative,index=mean_returns.index,columns=['Allocation'])

portfolio_agg_std_dev, portfolio_agg_return = portfolio_annualised_performance(weight_aggressive, mean_returns, cov_matrix)
portfolio_agg_sharpe = (portfolio_agg_return - risk_free_rate) / portfolio_agg_std_dev
agg_allocation = pd.DataFrame(weight_aggressive,index=mean_returns.index,columns=['Allocation'])
latest_prices = get_latest_prices(maindata) # for calculate number of stock unit to purchase

# Output the portfolio return and standard deviation of the Minimum Volatility Portfolio
print ("-"*80)
print ("Conservative Portfolio Performance\n")
print ("Expected annual return:", str(round(portfolio_con_return*100,2)) + "%")
print ("Annual volatility:", str(round(portfolio_con_std_dev*100,2)) + "%")
print ("Sharpe Ratio:", round(portfolio_con_sharpe,2))
print ("\n")
print("Asset Allocation\n")
print (con_allocation)
da = DiscreteAllocation(dict(zip(mean_returns.index, weight_conservative)), latest_prices, total_portfolio_value=Investment_money)
allocation, leftover = da.lp_portfolio()
print("Number of stock unit to purchase:", allocation)
print("Cash remaining: $",round(leftover,2))

# Output the portfolio return and standard deviation of the Minimum Volatility Portfolio
print ("-"*80)
print ("Aggressive Portfolio Performance\n")
print ("Expected annual return:", str(round(portfolio_agg_return*100,2)) + "%")
print ("Annual volatility:", str(round(portfolio_agg_std_dev*100,2)) + "%")
print ("Sharpe Ratio:", round(portfolio_agg_sharpe,2))
print ("\n")
print("Asset Allocation\n")
print (agg_allocation)
da = DiscreteAllocation(dict(zip(mean_returns.index, weight_aggressive)), latest_prices, total_portfolio_value=Investment_money)
allocation, leftover = da.lp_portfolio()
print("Number of stock unit to purchase:", allocation)
print("Cash remaining: $",round(leftover,2))


mu = mean_historical_return(maindata)
S = CovarianceShrinkage(maindata).ledoit_wolf()

# max sharpe ratio
ef_max_sharpe = EfficientFrontier(mu, S)
ef_max_sharpe.max_sharpe()

# min volatility
ef_min_vol = EfficientFrontier(mu, S)
ef_min_vol.min_volatility()


print ("-"*80)
print("Minimum Volatility Portfolio Performance")
print()
print(ef_min_vol.portfolio_performance(verbose=True))
print()
print("Asset Allocation")
print(pd.Series(ef_min_vol.clean_weights()))
da = DiscreteAllocation(ef_min_vol.clean_weights(), latest_prices, total_portfolio_value=Investment_money)
allocation, leftover = da.lp_portfolio()
print("Number of stock unit to purchase:", allocation)
print("Cash remaining: $",round(leftover,2))


print ("-"*80)
print("Maximum Sharpe Ratio Portfolio Performance")
print()
print(ef_max_sharpe.portfolio_performance(verbose=True))
print()
print("Asset Allocation")
print(pd.Series(ef_max_sharpe.clean_weights()))
da = DiscreteAllocation(ef_max_sharpe.clean_weights(), latest_prices, total_portfolio_value=Investment_money)
allocation, leftover = da.lp_portfolio()
print("Number of stock unit to purchase:", allocation)
print("Cash remaining: $",round(leftover,2))


C:\Users\sekso\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


--------------------------------------------------------------------------------
Conservative Portfolio Performance

Expected annual return: 9.45%
Annual volatility: 8.36%
Sharpe Ratio: 0.89


Asset Allocation

      Allocation
FTLS         0.3
QQQ          0.2
TIP          0.5
Number of stock unit to purchase: {'FTLS': 301, 'QQQ': 29, 'TIP': 197}
Cash remaining: $ 56.97
--------------------------------------------------------------------------------
Aggressive Portfolio Performance

Expected annual return: 14.45%
Annual volatility: 13.69%
Sharpe Ratio: 0.91


Asset Allocation

      Allocation
FTLS         0.2
QQQ          0.5
TIP          0.3
Number of stock unit to purchase: {'FTLS': 201, 'QQQ': 73, 'TIP': 118}
Cash remaining: $ 11.79
--------------------------------------------------------------------------------
Minimum Volatility Portfolio Performance

Expected annual return: 5.4%
Annual volatility: 5.2%
Sharpe Ratio: 0.64
(0.05351098696159767, 0.051980164251878125, 0.64468797749